Vamos a construir VGGNET para CIFAR10

VGG16-BN-CIFAR:

16 capas con parámetros entrenables (13 convoluciones + 3 lineales).

Se divide en bloques:

- 2 Bloques  (Conv + BatchNorm + Relu) x 2 -> MaxPool2

- 3 Bloques  (Conv + BatchNorm + Relu) x 3 -> MaxPool2

- Classificador Flatten → [Linear → ReLU → Dropout] ×2 → Linear

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from datetime import datetime
from torchmetrics.classification import MulticlassPrecision, MulticlassRecall, MulticlassF1Score

TENSORBOARD_EXP = f"runs/cifar10_cnn_step20_lr0015_{datetime.now().strftime('%Y%m%d-%H%M%S')}"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.manual_seed(3)

transform = transforms.ToTensor()

train_full = datasets.CIFAR10("./data", train=True, download=True, transform=transform)


#Obtenemos el dataset train completo
loader_train = DataLoader(train_full, batch_size=len(train_full), shuffle=False)                       
#Obtenemos un batch de datos
imgs, _ = next(iter(loader_train))             
imgs = imgs.to(torch.float64)
print(imgs.shape) # [50000,3,32,32]

CIFAR10_MEAN = imgs.mean(dim=(0,2,3))
CIFAR10_STD  = imgs.std(dim=(0,2,3))
print(CIFAR10_MEAN, CIFAR10_STD)


cuda
torch.Size([50000, 3, 32, 32])
tensor([0.4914, 0.4822, 0.4465], dtype=torch.float64) tensor([0.2470, 0.2435, 0.2616], dtype=torch.float64)


In [2]:

#CIFAR10_MEAN = torch.tensor([0.4914, 0.4822, 0.4465])
#CIFAR10_STD  = torch.tensor([0.2470, 0.2435, 0.2616])



#Creamos el transform para data augmentation y normalización de TRAIN_SET

train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN.tolist(), CIFAR10_STD.tolist())
    
])

#Creamos el transform para normalización de TRAIN_SET y EVAL sin aug
no_aug = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN.tolist(), CIFAR10_STD.tolist())
])

# Re-creamos los datasets con el nuevo transform + uno nuevo para validacion y que no pase por aug
train_full_aug = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_tf)

train_full_no_aug = datasets.CIFAR10(root="./data", train=True, download=True, transform=no_aug)

test_set = datasets.CIFAR10(root="./data", train=False, download=True, transform=no_aug)


#Comprobamos que los datos estan normalizados
check_loader_train_full = DataLoader(train_full_no_aug, batch_size=len(train_full_no_aug), shuffle=False) #Dataset completo para calcular la media y la desviación estándar de los datos ya normalizados
x, _ = next(iter(check_loader_train_full))
mean_check = x.mean(dim=(0, 2, 3))
std_check  = x.std(dim=(0, 2, 3))
print("Mean Appx: 0):", mean_check)
print("Std Appx: 1):", std_check)
assert mean_check.abs().max() < 0.05
assert (std_check - 1).abs().max() < 0.05



#Dividimos el dataset en train y validation para no_aug
train_set, val_set = torch.utils.data.random_split(generator=torch.Generator().manual_seed(3), dataset=train_full_no_aug, lengths=[40000, 10000])

#Dividimos el dataset en train y validation para AUG
train_set_aug, val_set_aug = torch.utils.data.random_split(generator=torch.Generator().manual_seed(3), dataset=train_full_aug, lengths=[40000, 10000])

#Comprobamos que el dataset se ha dividido correctamente
print(len(train_set), len(val_set))
assert train_set.indices == train_set_aug.indices
assert val_set.indices   == val_set_aug.indices


#Nuestros loaders para entrenar, validar y testear, ya normalizados.

#Vamos a mejorar rendimiento de la GPU

NUM_WORKERS = 4

loader_train = DataLoader(train_set_aug, batch_size=128, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, prefetch_factor=2) #Barajamos porque es train y mejora la generalización
loader_val = DataLoader(val_set, batch_size=256, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, prefetch_factor=2)
loader_test = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, prefetch_factor=2)








Mean Appx: 0): tensor([ 6.3542e-08, -1.9292e-08, -4.8446e-08])
Std Appx: 1): tensor([1.0000, 1.0000, 1.0000])
40000 10000


In [3]:
# VGG16-BN-CIFAR — 13 conv (con BN) + 3 FC (con Dropout)
import torch
import torch.nn as nn

def get_flatten_size(model_features, input_shape=(1, 3, 32, 32)):
    with torch.no_grad():
        x = torch.zeros(input_shape)
        out = model_features(x)
        return out.view(out.size(0), -1).size(1)

class VGG16_BN_CIFAR(nn.Module):
    def __init__(self, num_classes=10, p_drop=0.5):
        super().__init__()
        self.features = nn.Sequential(

            #==============================================
            # Block 1: 3->64, 32->16
            nn.Conv2d(3,   64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64,  64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2),
            #==============================================

            #==============================================
            # Block 2: 64->128, 16->8
            nn.Conv2d(64,  128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2),
            #==============================================

            #============================================== 
            # Block 3: 128->256, 8->4 (tres conv)
            nn.Conv2d(128, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(256), 
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2),
            #==============================================

            #==============================================
            # Block 4: 256->512, 4->2 (tres conv)
            nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.MaxPool2d(2),
            #==============================================

            #==============================================
            # Block 5: 512->512, 2->1 (tres conv)
            nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            
            nn.MaxPool2d(2),
            #==============================================
        )

        flat = get_flatten_size(self.features, input_shape=(1,3,32,32))  # esperado: 512*1*1 = 512

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat, 512), nn.ReLU(inplace=True),
            nn.Dropout(p=p_drop),
            nn.Linear(512, 512),  nn.ReLU(inplace=True),
            nn.Dropout(p=p_drop),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x
        

#GPU
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high") 

# Instancia + sanity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16_BN_CIFAR(p_drop=0.5).to(device)

with torch.no_grad():
    xb = torch.randn(4, 3, 32, 32, device=device)
    yb = model(xb)
print("Input:", xb.shape, "| Output:", yb.shape)  # [4, 10]
print("Params totales:", sum(p.numel() for p in model.parameters()))


/root/.pyenv/versions/dlvs/lib/python3.12/site-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


Input: torch.Size([4, 3, 32, 32]) | Output: torch.Size([4, 10])
Params totales: 15249354


Vamos a definir hparams

In [4]:
#TorchMetrics
NUM_CLASSES = 10  # CIFAR-10

precision_metric = MulticlassPrecision(num_classes=NUM_CLASSES, average="macro").to(device)
recall_metric    = MulticlassRecall(num_classes=NUM_CLASSES, average="macro").to(device)
f1_metric        = MulticlassF1Score(num_classes=NUM_CLASSES, average="macro").to(device)

#Definimos los hparams
EPOCHS = 80  # número de pasadas por el dataset
lr = 0.1
momentum = 0.9
#Definimos la funcion de perdida y el optimizador
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
weight_decay=5e-4
#lr es la tasa de aprendizaje, momentum es el factor de inercia, es decir, cuanto se mueve el optimizador en la direccion del gradiente y 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


#DECLARAMOS EL SCHEDULER
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, threshold=1e-3)
lrs = []

model.to(device)  # mueve el modelo a GPU
model.train()     # pone el modelo en modo entrenamiento (activa dropout, etc.)



VGG16_BN_CIFAR(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, paddin

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=TENSORBOARD_EXP)  # <-- añadido

def evaluate(model, loader, device, criterion):
    model.eval()
    total, correct, total_loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            logits = model(images)
            loss = criterion(logits, labels)
            total_loss += loss.item() * images.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / total, correct / total


train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
lrs = []

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in loader_train:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total
    val_loss, val_acc = evaluate(model, loader_val, device, criterion)

    precision_metric.reset()
    recall_metric.reset()
    f1_metric.reset()

    with torch.no_grad():
        for images, labels in loader_val:
            images, labels = images.to(device), labels.to(device)
            preds = model(images).argmax(dim=1)
            precision_metric.update(preds, labels)
            recall_metric.update(preds, labels)
            f1_metric.update(preds, labels)

    precision_val = precision_metric.compute().item()
    recall_val    = recall_metric.compute().item()
    f1_val        = f1_metric.compute().item()

    writer.add_scalar("Val/Precision_macro", precision_val, epoch)
    writer.add_scalar("Val/Recall_macro",    recall_val,  epoch)
    writer.add_scalar("Val/F1_macro",        f1_val,      epoch)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(
        f"Época {epoch}/{EPOCHS} | "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
    )

    lrs.append(optimizer.param_groups[0]['lr'])
    scheduler.step(val_loss)

    #Escribimos los resultados de cada epoch en tensorboard
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val",  val_loss,  epoch)
    writer.add_scalar("Acc/train",  train_acc,  epoch)
    writer.add_scalar("Acc/val",   val_acc,   epoch)
    writer.add_scalar("LR", optimizer.param_groups[0]['lr'], epoch)
    writer.flush()  # <-- fuerza escritura inmediata para ver en tiempo real

#Evaluamos el modelo en el test set y añadimos los resultados a tensorboard
test_loss, test_acc = evaluate(model, loader_test, device, criterion)

hparams = {
    'model': model.__class__.__name__,
    'seed': 3,
    'optimizer': optimizer.__class__.__name__,
    'lr_init': float(lr),
    'momentum': float(momentum),
    'batch_size': int(loader_train.batch_size),
    'weight_decay': float(optimizer.param_groups[0].get('weight_decay', 0.0)),
    'scheduler': type(scheduler).__name__,
}

metrics = {
    'metrics/test_acc': float(test_acc),
    'metrics/test_loss': float(test_loss),
    'metrics/val_acc_last': float(val_accuracies[-1]),
    'metrics/val_loss_last': float(val_losses[-1]),
    'metrics/train_acc_last': float(train_accuracies[-1]),
    'metrics/train_loss_last': float(train_losses[-1]),
}

writer.add_hparams(hparams, metrics)
writer.close()



Época 1/80 | Train Loss: 2.0784 | Train Acc: 0.1842 | Val Loss: 2.0352 | Val Acc: 0.1913
Época 2/80 | Train Loss: 1.9994 | Train Acc: 0.2125 | Val Loss: 1.9119 | Val Acc: 0.2586
Época 3/80 | Train Loss: 1.9201 | Train Acc: 0.2828 | Val Loss: 1.7730 | Val Acc: 0.3361
Época 4/80 | Train Loss: 1.7744 | Train Acc: 0.3477 | Val Loss: 1.6908 | Val Acc: 0.3617
Época 5/80 | Train Loss: 1.6612 | Train Acc: 0.4154 | Val Loss: 1.5906 | Val Acc: 0.4506
Época 6/80 | Train Loss: 1.5413 | Train Acc: 0.4935 | Val Loss: 1.6568 | Val Acc: 0.4499
Época 7/80 | Train Loss: 1.4486 | Train Acc: 0.5383 | Val Loss: 1.6919 | Val Acc: 0.4621
Época 8/80 | Train Loss: 1.3527 | Train Acc: 0.6125 | Val Loss: 1.3550 | Val Acc: 0.6191
Época 9/80 | Train Loss: 1.2760 | Train Acc: 0.6677 | Val Loss: 1.5226 | Val Acc: 0.5787
Época 10/80 | Train Loss: 1.2293 | Train Acc: 0.6953 | Val Loss: 1.3716 | Val Acc: 0.6377
Época 11/80 | Train Loss: 1.1754 | Train Acc: 0.7236 | Val Loss: 1.3700 | Val Acc: 0.6547
Época 12/80 | Train

In [ ]:
# Exportar modelo a ONNX (compatibilidad PyTorch <=2.2)
import torch.onnx

model.eval()
model_cpu = model.to('cpu').to(memory_format=torch.contiguous_format)
dummy_input = torch.randn(1, 3, 32, 32)

torch.onnx.export(
    model_cpu,
    dummy_input,
    "cifar10cnn.onnx",
    input_names=["input"],
    output_names=["logits"],
    opset_version=13,
    do_constant_folding=True,
    dynamo=False,  # fuerza el exportador estable
    training=torch.onnx.TrainingMode.EVAL
)

print("Modelo exportado correctamente a cifar10cnn.onnx")
